In [ ]:
# functions to create the boards and dataset
import random
import json
import numpy as np
import matplotlib.pyplot as plt
# import libraries for plotting
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
# import the library for ListedColormap
from matplotlib.colors import ListedColormap, Normalize
from genDSL_helpers import plot_riddle

################# PARAMETERS TO CHANGE ####################
dataset_folder = './10_pr_cc_test' # will make 3 subfolders in this folder: pixel_removal, color_change, combined

#total number of puzzles (meta samples)
dataset_size = 10 # per dataset
plot_items = 3 # number of puzzles to plot
min_num_colors_per_task = 3 # minimum number of colors per task (e.g., on combined there would be 3 colors for px removal and 3 colors for color change and 3 random colors)

#number of training boards and testing boards 0 for random
meta_trn_size = 5
meta_tst_size = 1 # always 1

# grid size (width and height); if -1 then random
# if using a fixed grid size, 7x7 is a recommended minimum, because it becomes very slow
all_items_same_grid_size = True  # only applies if grid_size is -1
grid_size_width = 8
grid_size_height = 8
min_grid_size = 8 # applies if grid_size is -1

In [ ]:

# create a function to add a point to a grid
def add_point(grid, color, x, y):
        grid[x][y] = color
        return grid

def is_corner(grid, x, y):
        '''
        Returns true if the point is a corner of the grid
        '''
        if (x == 0 and y == 0) or (x == 0 and y == len(grid[0])-1) or (x == len(grid)-1 and y == 0) or (x == len(grid)-1 and y == len(grid[0])-1):
                return True
        return False

def is_corner_adjacent(grid, x, y):
        '''
        Returns true if the point is adjacent to a corner of the grid (either up or down or left or right)
        '''
        if (x == 0 and y == 1) or (x == 1 and y == 0) or (x == 0 and y == len(grid[0])-2) or (x == 1 and y == len(grid[0])-1) or (x == len(grid)-2 and y == 0) or (x == len(grid)-1 and y == 1) or (x == len(grid)-2 and y == len(grid[0])-1) or (x == len(grid)-1 and y == len(grid[0])-2):
                return True
        return False

# create a function to add a point to a random location in a grid that is not already occupied
def add_point_random(grid, color_to_add, bg_color, exclude_corners=False, exclude_adjacent_corners=False):
        if color_to_add!=bg_color:
                # get the number of cells in the grid
                grid_size = grid.size
                unoccupied_cells = color_count(grid, bg_color)
                added_point = False
                error=False
                while (not added_point and unoccupied_cells > 0) and not error:
                        unoccupied_cells = color_count(grid, bg_color)
                        if unoccupied_cells == 0:
                                error=True
                        if unoccupied_cells <= 4 and exclude_corners:
                                error=True
                        if unoccupied_cells <= 12 and exclude_adjacent_corners:
                                error=True

                        # get a random number between 0 and the number of unoccupied cells
                        random_cell = random.randint(0, unoccupied_cells)
                        # create a counter to count the number of unoccupied cells
                        unoccupied_cell_count = 0
                        # loop through the grid

                        for x in range(grid.shape[0]):
                                for y in range(grid.shape[1]):
                                        
                                        # if the cell is not occupied
                                        if grid[x][y] == bg_color:
                                                # if the cell is not a corner
                                                if not added_point and (not exclude_corners or (is_corner(grid, x, y) and not exclude_corners) or not is_corner(grid, x, y)):
                                                        # check for adjacent corners
                                                        if not added_point and (not exclude_adjacent_corners or (is_corner_adjacent(grid, x, y) and not exclude_adjacent_corners) or not is_corner_adjacent(grid, x, y)):
                                                                
                                                                # if the unoccupied cell counter is equal to the random cell
                                                                if unoccupied_cell_count >= random_cell:
                                                                        # add the point to the grid
                                                                        grid = add_point(grid, color_to_add, x, y)
                                                                        added_point = True
                                                                        break
                                                # increment the unoccupied cell counter
                                                unoccupied_cell_count += 1
                                if added_point:
                                        break
        return grid, not error

def color_count(grid, color_to_count):
        color_count = 0
        for row in grid:
                for cell in row:
                        if cell == color_to_count:
                                color_count += 1
        return color_count

def color_count_excluding(grid, color_list_to_exclude):
        color_count = 0
        for row in grid:
                for cell in row:
                        if cell not in color_list_to_exclude:
                                color_count += 1
        return color_count


# create a function to create a board that has x number of random color points on the specified background
def create_board(color_bg, color_fg, x, y, points_max, color_fg2=0, rand_points=0, color_fg3=0, exclude_colors_in_corners=False, exclude_colors_in_adjacent_corners=False):
        #print(f"create board {color_bg} {color_fg} {x} {y} {points_max} {color_fg2} {rand_points} {color_fg3} {exclude_colors_in_corners} {exclude_colors_in_adjacent_corners}")
        # create a grid of the specified size
        grid = np.full((x,y), color_bg, dtype=np.int8)
        if points_max <= 2:
                points_max == 3
        # set a random number of points to the specified color
        num_points = max(np.random.randint(1,points_max), 2)
        point_count = 0
        point_count2 = 0
        point_count3 = 0
        rand_point_count = 0
        should_continue = True
        while point_count < num_points or (color_fg2 > 0 and point_count2 < num_points) or (rand_points > 0 and rand_point_count < rand_points) or (color_fg3 > 0 and point_count3 < num_points):
                grid, should_continue = add_point_random(grid, color_fg, color_bg, exclude_corners=exclude_colors_in_corners, exclude_adjacent_corners=exclude_colors_in_adjacent_corners)
                if color_fg2 != color_bg and should_continue:
                        grid, should_continue = add_point_random(grid, color_fg2, color_bg, exclude_corners=exclude_colors_in_corners, exclude_adjacent_corners=exclude_colors_in_adjacent_corners)
                if color_fg3 != color_bg and should_continue:                
                        grid, should_continue = add_point_random(grid, color_fg3, color_bg, exclude_corners=exclude_colors_in_corners, exclude_adjacent_corners=exclude_colors_in_adjacent_corners)
                if rand_point_count <= rand_points and should_continue:
                        rand_color = np.random.randint(1, 9)
                        while rand_color in [color_bg, color_fg, color_fg2, color_fg3]:
                                rand_color = np.random.randint(1, 9)
                        grid, should_continue = add_point_random(grid, rand_color, color_bg, exclude_corners=exclude_colors_in_corners, exclude_adjacent_corners=exclude_colors_in_adjacent_corners)
                if should_continue:
                        point_count = color_count(grid, color_fg)
                        point_count2 = color_count(grid, color_fg2)
                        point_count3 = color_count(grid, color_fg3)
                        rand_point_count = color_count_excluding(grid, [color_bg, color_fg, color_fg2, color_fg3])
                else:
                        break
        return grid, not should_continue

def create_pixel_removal_riddle(num_input_output_grids, num_rows = -1, num_cols = -1, min_num_colors = 2):
    '''
    create a simple riddle that requires the user to remove a pixel from a grid
    the pixel to remove will be in one of the corners of the input grid
    '''
    input_grids = []
    output_grids = []
    params = {}
    params['input'] = []
    params['output'] = []
    color_bg = 0
    color_fg = np.random.randint(1,10)
    for i in range(num_input_output_grids):
            # create the input grid
            error=True
            while error:
                # set the parameters for the input and output grids
                x = np.random.randint(min_grid_size,31) if num_cols == -1 else num_cols
                y = np.random.randint(min_grid_size,31) if num_rows == -1 else num_rows
                # pick a random number of points proportional to the size of the grid
                points = np.random.randint(1, x*y/4)
                current_board, error = create_board(color_bg, color_fg, x, y, max(points, min_num_colors), 0, max(points/2, min_num_colors), 0, True, True)
                input_grids.append(current_board)

            # choose one of the corners to randomly change to the foreground color
            corner = np.random.randint(0,4)
            if corner == 0:
                    input_grids[-1][0][0] = color_fg
            elif corner == 1:
                    input_grids[-1][0][-1] = color_fg
            elif corner == 2:
                    input_grids[-1][-1][0] = color_fg
            elif corner == 3:
                    input_grids[-1][-1][-1] = color_fg

            # the output grid is the same but the points with the foreground color are removed
            output_grids.append(np.where(input_grids[-1] == color_fg, color_bg, input_grids[-1]))

            # append the parameters to the params object
            params['input'].append({'color_bg': color_bg, 'color_fg': color_fg, 'points': points, 'x': x, 'y': y})
            params['output'].append({'color_bg': color_bg, 'color_fg': color_fg, 'points': points, 'x': x, 'y': y})
    return input_grids, output_grids, params

def create_color_change_riddle(num_input_output_grids, num_rows = -1, num_cols = -1, min_num_colors = 2):
        '''
        Create a simple riddle that requires changing a pixel color from intput to output
        The color to change is indicated by a pixel in a corner.  Then the pixel adjacent to that pixel is the color to change to.
        The output board contains all the same pixels with the color changed from input.
        '''
        input_grids = []
        output_grids = []
        params = {}
        params['input'] = []
        params['output'] = []
        color_bg = 0
        random_colors = get_n_random_colors(2)
        color_fg = random_colors[0]
        color_fg2 = random_colors[1]
        for i in range(num_input_output_grids):
                error=True
                while error:
                        # set the parameters for the input and output grids
                        x = np.random.randint(min_grid_size,31) if num_cols == -1 else num_cols
                        y = np.random.randint(min_grid_size,31) if num_rows == -1 else num_rows
                        # pick a random number of points proportional to the size of the grid
                        points = np.random.randint(1, x*y/4)
                        current_board, error = create_board(color_bg, color_fg, x, y, max(points, min_num_colors), color_fg2, max(points/2, min_num_colors), 0, True, True)
                        input_grids.append(current_board)

                # choose one of the corners to randomly change to the foreground color
                corner = np.random.randint(0,4)
                if corner == 0:
                        input_grids[-1][0][0] = color_fg
                        # make the adjacent pixel the color to change to random to choose which adjacent pixel to change to
                        change_to = np.random.randint(0,2) # 0 = right or left, 1 = down or up
                        if change_to == 0:
                                input_grids[-1][0][1] = color_fg2
                        else:
                                input_grids[-1][1][0] = color_fg2
                elif corner == 1:
                        input_grids[-1][0][-1] = color_fg
                        # make the adjacent pixel the color to change to random to choose which adjacent pixel to change to
                        change_to = np.random.randint(0,2)
                        if change_to == 0:
                                input_grids[-1][0][-2] = color_fg2
                        else:
                                input_grids[-1][1][-1] = color_fg2
                elif corner == 2:
                        input_grids[-1][-1][0] = color_fg
                        # make the adjacent pixel the color to change to random to choose which adjacent pixel to change to
                        change_to = np.random.randint(0,2)
                        if change_to == 0:
                                input_grids[-1][-1][1] = color_fg2
                        else:
                                input_grids[-1][-2][0] = color_fg2
                elif corner == 3:
                        input_grids[-1][-1][-1] = color_fg
                        # make the adjacent pixel the color to change to random to choose which adjacent pixel to change to
                        change_to = np.random.randint(0,2)
                        if change_to == 0:
                                input_grids[-1][-1][-2] = color_fg2
                        else:
                                input_grids[-1][-2][-1] = color_fg2

                # the output grid is the same but the points with the fg changed to fg2
                output_grids.append(np.where(input_grids[-1] == color_fg, color_fg2, input_grids[-1]))

                # append the parameters to the params object
                params['input'].append({'color_bg': color_bg, 'color_fg': color_fg, 'points': points, 'x': x, 'y': y})
                params['output'].append({'color_bg': color_bg, 'color_fg': color_fg, 'points': points, 'x': x, 'y': y})
        return input_grids, output_grids, params

def get_distinct_colors(grid):
    '''
    Returns a list of distinct colors in the grid
    '''
    # Find all colors that are in the board
    colors = set(np.unique(grid).tolist())
    return list(colors)

def get_n_random_colors(n):
    '''
    Returns a list of n random colors
    '''
    colors = []
    while len(colors) < n:
        color = np.random.randint(1,10)
        if color not in colors:
            colors.append(color)
    return colors

def create_combined_riddle(num_input_output_grids, num_rows = -1, num_cols = -1, min_num_colors = 2):
        '''
        combine both pixel removal and color change riddles
        '''
        input_grids = []
        output_grids = []
        params = {}
        params['input'] = []
        params['output'] = []
        color_bg = 0
        random_colors = get_n_random_colors(3)
        color_fg = random_colors[0]
        color_fg2 = random_colors[1]
        color_to_remove = random_colors[2]

        for i in range(num_input_output_grids):
                error=True
                while error:
                        # set the parameters for the input and output grids
                        x = np.random.randint(min_grid_size,31) if num_cols == -1 else num_cols
                        y = np.random.randint(min_grid_size,31) if num_rows == -1 else num_rows
                        # pick a random number of points proportional to the size of the grid
                        points = np.random.randint(1, x*y/4)
                        current_board, error = create_board(color_bg, color_fg, x, y, max(points, min_num_colors), color_fg2, max(points/2, min_num_colors), color_to_remove, True, True)
                        input_grids.append(current_board)

                # choose one of the corners to randomly change to the foreground color
                corner = np.random.randint(0,4)
                if corner == 0:
                        input_grids[-1][0][0] = color_fg
                        # make the adjacent pixel the color to change to random to choose which adjacent pixel to change to
                        change_to = np.random.randint(0,2) # 0 = right or left, 1 = down or up
                        if change_to == 0:
                                input_grids[-1][0][1] = color_fg2
                        else:
                                input_grids[-1][1][0] = color_fg2
                elif corner == 1:
                        input_grids[-1][0][-1] = color_fg
                        # make the adjacent pixel the color to change to random to choose which adjacent pixel to change to
                        change_to = np.random.randint(0,2)
                        if change_to == 0:
                                input_grids[-1][0][-2] = color_fg2
                        else:
                                input_grids[-1][1][-1] = color_fg2
                elif corner == 2:
                        input_grids[-1][-1][0] = color_fg
                        # make the adjacent pixel the color to change to random to choose which adjacent pixel to change to
                        change_to = np.random.randint(0,2)
                        if change_to == 0:
                                input_grids[-1][-1][1] = color_fg2
                        else:
                                input_grids[-1][-2][0] = color_fg2
                elif corner == 3:
                        input_grids[-1][-1][-1] = color_fg
                        # make the adjacent pixel the color to change to random to choose which adjacent pixel to change to
                        change_to = np.random.randint(0,2)
                        if change_to == 0:
                                input_grids[-1][-1][-2] = color_fg2
                        else:
                                input_grids[-1][-2][-1] = color_fg2

                # the output grid is the same but the points with the fg changed to fg2
                output_grids.append(np.where(input_grids[-1] == color_fg, color_fg2, input_grids[-1]))

                # now do the pixel removal. place color_fg in a corner
                corner2 = np.random.randint(0,4)
                while corner2 == corner:
                        corner2 = np.random.randint(0,4)
                if corner2 == 0:
                        input_grids[-1][0][0] = color_to_remove
                elif corner2 == 1:
                        input_grids[-1][0][-1] = color_to_remove
                elif corner2 == 2:
                        input_grids[-1][-1][0] = color_to_remove
                elif corner2 == 3:
                        input_grids[-1][-1][-1] = color_to_remove

                # remove the color from the output grid
                output_grids[-1] = np.where(output_grids[-1] == color_to_remove, color_bg, output_grids[-1])

                # append the parameters to the params object
                params['input'].append({'color_bg': color_bg, 'color_fg': color_fg, 'points': points, 'x': x, 'y': y})
                params['output'].append({'color_bg': color_bg, 'color_fg': color_fg, 'points': points, 'x': x, 'y': y})
        return input_grids, output_grids, params


In [ ]:
# import library for a progress indicator
from tqdm import tqdm

riddle_creation_functions = [create_pixel_removal_riddle, create_color_change_riddle, create_combined_riddle]
#riddle_creation_functions = [create_combined_riddle]

riddle_labels = ["pixel_removal", "color_change", "combined_pr_cc"]
#riddle_labels = ["combined_pr_cc"]
if all_items_same_grid_size:
    rows = np.random.randint(min_grid_size,31) if grid_size_height == -1 else grid_size_height
    cols = np.random.randint(min_grid_size,31) if grid_size_width == -1 else grid_size_width
else:
    rows = -1
    cols = -1

for i in range(len(riddle_creation_functions)):
    riddle_creation_function= riddle_creation_functions[i]
    plot_count = 0
    print(f"Creating {riddle_labels[i]} riddles...")
    # init progress bar
    pbar = tqdm(total=dataset_size, position=0)

    for c in range(dataset_size):
        number_test_items = np.random.randint(3,7) if meta_trn_size==0 else meta_trn_size+1
        input_grids, output_grids, params = riddle_creation_function(number_test_items, rows, cols, min_num_colors_per_task)
        pbar.update(1)

        # set the train and test data; train is all but the last item, test is the last item
        train_input_grids = input_grids[:-1]
        train_output_grids = output_grids[:-1]
        test_input_grids = input_grids[-1:]
        test_output_grids = output_grids[-1:]

        json_obj = {}
        json_obj['train'] = []
        json_obj['test'] = []
        # train
        for io_pair in range(len(train_input_grids)):
            a = {
                'input': train_input_grids[io_pair].tolist(),
                'output': train_output_grids[io_pair].tolist()
            }
            json_obj['train'].append(a)

        a = {'input': test_input_grids[0].tolist(),
                'output': test_output_grids[0].tolist()    }
        json_obj['test'].append(a)
        if not os.path.exists(dataset_folder):
            os.makedirs(dataset_folder)
        output_folder = os.path.join(dataset_folder, riddle_labels[i])
        # verify folder exists ./dungeon_dataset/
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        #make a new filename that is 8 characters long and represents a random hex number
        # characters betwee 0-9 and a-f
        string = '0123456789abcdef'
        filename = ''.join(random.choice(string) for t in range(8)) + '.json'
        # write the json object to a file
        with open(os.path.join(output_folder, filename), 'w') as outfile:
            json.dump(json_obj, outfile)
        if plot_count < plot_items:
            # plot the riddle
            plot_riddle(train_input_grids, train_output_grids, test_input_grids, test_output_grids)
            plot_count += 1

    # dataset generated